1. Get a somewhat parsed full-text from xDD team, preferably had section label, citation label, etc.
2. Query-based text summarization (Similar to Petal's AI table)
3. Sort into support and counter arguments based on semantics
4. Return sorted evidences 

In [3]:
from datasets import load_dataset

dataset = load_dataset("covid_qa_deepset")


Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

Dataset covid_qa_deepset downloaded and prepared to /root/.cache/huggingface/datasets/covid_qa_deepset/covid_qa_deepset/1.0.0/fb886523842e312176f92ec8e01e77a08fa15a694f5741af6fc42796ee9c8c46. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 2019
    })
})

In [ ]:
# pip install -q transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-7b1"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

In [ ]:
pip install triton

### PDF Parser

In [ ]:
# from tqdm.autonotebook import tqdm
# import pathlib
# from pdf_parser import parse_pdf
# pdfs = pathlib.Path('data/').glob('*.pdf')

# for i, pdf in tqdm(enumerate(pdfs)):
#     paper = parse_pdf(pdf)
#     paper.save(f'data/parsed/{i}.pkl.gz')

In [ ]:
from pdf_parser import Paper
from transformers import AutoTokenizer, AutoModel

# Load parsed PDF
p = Paper.load("data/parsed/0.pkl.gz")
print(p)

In [2]:
len(text)

31061

### Long T-5
- https://huggingface.co/google/long-t5-tglobal-base
- https://huggingface.co/google/long-t5-tglobal-xl
- https://huggingface.co/google/long-t5-tglobal-large


Relevant but not directly useful... also buggy

In [ ]:
from transformers import AutoTokenizer, LongT5Model

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
model = LongT5Model.from_pretrained("google/long-t5-tglobal-base")

inputs = tokenizer("Based on this context", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state


### Run GPT summarization with `compress`

In [ ]:
from backend import compress, get_answer

# s = compress(p.chunks)
# short_text = '\n'.join(s)

# with open('tmp/tmp.txt', 'w') as f:
#     f.write(short_text)


### GPT3.5 baseline

In [ ]:
# write short_text to short_text.txt

with open("tmp/tmp.txt", "r") as f:
    short_text = f.read()

In [ ]:
get_answer(context=short_text, question="What is QSPR modeling workflow?")


### Subset to small test toy example

### Long-T5
https://huggingface.co/google/long-t5-tglobal-base

### mt0

 | Params | Checkpoint |
 | ------ | ---------- |
 | 300M   | mt0-small  |
 | 580M   | mt0-base   |
 | 1.2B   | mt0-large  |
 | 3.7B   | mt0-xl     |
 | 13B    | mt0-xxl    |

### BloomZ

 | Params | Checkpoint  |
 | ------ | ----------- |
 | 560M   | bloomz-560m |
 | 1.1B   | bloomz-1b1  |
 | 1.7B   | bloomz-1b7  |
 | 3B     | bloomz-3b   |
 | 7.1B   | bloomz-7b1  |
 | 176B   | bloomz      |

In [ ]:
short_text


In [ ]:
# pip install -q transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

# checkpoint = "bigscience/bloomz"
checkpoint = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [ ]:
inputs = tokenizer.encode(
    "based on this article, what is QSPR modeling workflow? " + short_text,
    return_tensors="pt",
)
outputs = model.generate(inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

### m-t0

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "bigscience/mt0-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint, torch_dtype="auto", device_map="auto"
)

In [ ]:
inputs = tokenizer.encode(
    "based on this article, what is QSPR modeling workflow? " + short_text,
    return_tensors="pt",
)
outputs = model.generate(inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0]))

# References

### Paragraph segmentation

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="dennlinger/bert-wiki-paragraphs")
pipe("{First paragraph} [SEP] {Second paragraph}")

### Embedding from distilbert

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")


def get_sentence_embeddings(sentences, tokenizer, model):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings